In [28]:
import pandas as pd
import numpy as np
import sqlite3
import urllib
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [2]:
#driver.close()

In [2]:
# 드라이버 객체 생성
driver = webdriver.Chrome('chromedriver')

In [3]:
#### BeautifulSoup를 이용해 리팩토링 요망 ####
# 상품코드 수집
def 상품코드수집():
    url = 'https://www.meatbox.co.kr/fo/product/productListPage.do'
    params = {'categorySeq':10005,'articleCount':'1000'}
    url_params = '{}?{}'.format(url, urllib.parse.urlencode(params))
    driver.get(url_params)
    상품정보 = driver.find_elements_by_xpath("//li[@data-product-seq]")
    상품코드 = []
    for 상품 in 상품정보:
        상품코드.append(상품.get_attribute('data-product-seq'))
    return 상품코드

In [36]:
상품코드 = 상품코드수집()
코드 = 상품코드[40]
코드

'123502'

In [37]:
def 상세정보수집(seq):
    url = 'https://www.meatbox.co.kr/fo/product/productViewPage.do'
    params = {'productSeq':seq}
    url_params = '{}?{}'.format(url, urllib.parse.urlencode(params))
    driver.get(url_params)
    
    테이블 = driver.find_element_by_xpath("//table[@class='section_td']").find_elements_by_tag_name('tr')
    상세정보 = []
    for 레코드 in 테이블:
        속성 = 레코드.find_element_by_tag_name('th').text
        내용 = 레코드.find_element_by_tag_name('td').text
        상세정보.append([속성, 내용])
    return 상세정보

In [38]:
상세정보 = 상세정보수집(코드)

In [39]:
def 판매자정보수집(seq):
    url = 'https://www.meatbox.co.kr/fo/product/productViewPage.do'
    params = {'productSeq':seq}
    url_params = '{}?{}'.format(url, urllib.parse.urlencode(params))
    driver.get(url_params)
    
    #판매업체버튼 = driver.find_element_by_xpath("//button[@class='sale_info']")
    #판매업체버튼.click()
    #time.sleep(3)
    #테이블 = driver.find_element_by_xpath("//div[@class='pop_comm sale_pop']").find_element_by_xpath("div[@class='pop_cont']").find_elements_by_tag_name('tr')
    테이블 = driver.find_element_by_xpath("//div[@class='pop_comm sale_pop']").find_elements_by_tag_name('tr')
    판매자정보 = []
    for 레코드 in 테이블:
        속성 = 레코드.find_element_by_tag_name('th').get_attribute('innerHTML')
        내용 = 레코드.find_element_by_tag_name('td').get_attribute('innerHTML')
        판매자정보.append([속성, 내용])
        
    판매자정보[4][1] = 판매자정보[4][1].split('<strong>')[0]
    return 판매자정보

In [40]:
판매자정보 = 판매자정보수집(코드)

In [41]:
def 상품정보수집(seq):
    url = 'https://www.meatbox.co.kr/fo/product/productViewPage.do'
    params = {'productSeq':seq}
    url_params = '{}?{}'.format(url, urllib.parse.urlencode(params))
    driver.get(url_params)
    
    테이블 = driver.find_element_by_xpath("//div[@class='pop_comm deteil_over_pop _optionDetailInfoLayer']").find_element_by_xpath("//div[@class='table_wrap']").find_elements_by_tag_name('tr')
    상품정보 = []
    for 레코드 in 테이블:
        속성 = 레코드.find_element_by_tag_name('th').get_attribute('innerHTML')
        내용 = 레코드.find_element_by_tag_name('td').get_attribute('innerHTML')
        상품정보.append([속성, 내용])
    return 상품정보

In [42]:
상품정보 = 상품정보수집(코드)

In [43]:
판매자정보

[['대표자', '김창환'],
 ['상호', '(주)유명푸드'],
 ['전화', '031-754-4928'],
 ['이메일 주소', 'eumyeong@nate.com'],
 ['사업자등록번호', '2128178298'],
 ['사업장 소재지', '경기도 성남시 수정구 복정로 133 (복정동 645-2)&nbsp;복정동 2층']]

In [44]:
식별번호 = 상품정보[-1][-1]
상품정보

[['kg당 가격', '4,000원'],
 ['가공일', '2017-11-01'],
 ['유통기한', '2019-10-31'],
 ['최대', '22.80kg'],
 ['최소', '13.12kg'],
 ['평균', '17.46kg'],
 ['유통식별번호', '804002013266']]

In [45]:
상세정보

[['보관방법', '냉동'],
 ['축종', '수입소'],
 ['품목', '등갈비/빽립'],
 ['가공방법', '원물'],
 ['원산지', '뉴질랜드'],
 ['브랜드', '실버펀팜'],
 ['등급', 'COW'],
 ['배송정책', '무료배송'],
 ['판매대상', '사업자회원 전용상품'],
 ['EST NO', 'ME 26'],
 ['포장상태', '레이어포장']]

In [46]:
url = 'https://www.meatwatch.go.kr/cs/dst/bdi/selectDistbHistInfoDetail.do'
driver.get(url)

In [47]:
driver.find_element_by_id('searchWrd1').clear()
driver.find_element_by_id('searchWrd1').send_keys(식별번호)
driver.find_element_by_id('searchWrd1').submit()

In [48]:
기본정보, 원산지정보, 수입이력정보 = driver.find_elements_by_xpath("//div[@class='tform1']")

In [49]:
테이블 = 기본정보.find_elements_by_tag_name('tr')
기본정보 = []
for 레코드 in 테이블:
    속성들 = 레코드.find_elements_by_tag_name('th')
    내용들 = 레코드.find_elements_by_tag_name('td')
    for 속성, 내용 in zip(속성들, 내용들):
        속성 = 속성.get_attribute('innerHTML')
        내용 = 내용.get_attribute('innerHTML')
        기본정보.append([속성, 내용])

In [50]:
테이블 = 원산지정보.find_elements_by_tag_name('tr')
원산지정보 = []
for 레코드 in 테이블:
    속성들 = 레코드.find_elements_by_tag_name('th')
    내용들 = 레코드.find_elements_by_tag_name('td')
    for 속성, 내용 in zip(속성들, 내용들):
        속성 = 속성.get_attribute('innerHTML')
        내용 = 내용.get_attribute('innerHTML')
        원산지정보.append([속성, 내용])

In [51]:
테이블 = 수입이력정보.find_elements_by_tag_name('tr')
수입이력정보 = []
for 레코드 in 테이블:
    속성들 = 레코드.find_elements_by_tag_name('th')
    내용들 = 레코드.find_elements_by_tag_name('td')
    for 속성, 내용 in zip(속성들, 내용들):
        속성 = 속성.get_attribute('innerHTML')
        내용 = 내용.get_attribute('innerHTML')
        수입이력정보.append([속성, 내용])

In [52]:
# 냉동전환정보는 없을 수도 있음
try:
    냉동전환정보 = driver.find_element_by_xpath("//div[@class='grid1']")
    컬럼명 = 냉동전환정보.find_element_by_tag_name('thead').find_elements_by_tag_name('th')

    총정보 = []
    컬럼정보 = []
    for 컬럼 in 컬럼명:
        컬럼정보.append(컬럼.text.replace('\n',' '))
    총정보.append(컬럼정보)

    데이터 = 냉동전환정보.find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')
    for 레코드 in 데이터:
        임시 = []
        필드 = 레코드.find_elements_by_tag_name('td')
        for 컬럼 in 필드:
            임시.append(컬럼.text)
        총정보.append(임시)
except NoSuchElementException:
    print('냉동전환정보 없음')

냉동전환정보 없음


In [53]:
기본정보

[['이력번호', '804002013266'], ['선하증권번호', 'MAEU575151254'], ['수입쇠고기 품목명', '냉동쇠고기']]

In [54]:
원산지정보

[['원산지 국가', '뉴질랜드']]

In [55]:
수입이력정보

[['수출국 도축장', 'SILVER FERN FARMS LIMITED-FINEGAND'],
 ['수출국 도축일자', '2017-10-31 ~ 2018-03-05'],
 ['수출국 가공장', 'SILVER FERN FARMS LIMITED-FINEGAND'],
 ['수출국 가공일자', '2017-11-01 ~ 2018-03-06'],
 ['수출업체', 'SILVER FERN FARMS BEEF LIMITED'],
 ['수입업체', '(주)유명푸드'],
 ['수입일자', '2018-04-13'],
 ['부위명(코드)', '갈비(BF010)']]